In [ ]:
import pandas as pd
import math
import re
import os
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ------------------- Qdrant Bağlantısı -------------------

def connect_qdrant():
    return QdrantClient(
        url="http://localhost:6333",
        prefer_grpc=False,
        timeout=60.0
    )

# ------------------- Metin Yardımcıları -------------------

def normalize_text(text):
    return text.replace(" ", "").lower()

def split_into_sentences(text):
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]

def count_words(text):
    return len(text.split())

# ------------------- Chunkları Getir -------------------

def retrieve_chunks(collection_name, embed_model, query, k):
    client = connect_qdrant()
    vector = embed_model.encode([query], convert_to_numpy=True)[0].tolist()

    results = client.search(
        collection_name=collection_name,
        query_vector=vector,
        limit=k,
        with_payload=True
    )

    chunks = []
    for hit in results:
        chunks.append(hit.payload.get("original_text", ""))
    return chunks

# ------------------- Recall -------------------

def recall_metric(gt_sentence, chunks, use_contains=True):
    gt = normalize_text(gt_sentence)
    cleaned_chunks = [normalize_text(c) for c in chunks]

    if use_contains:
        return int(any(gt in chunk for chunk in cleaned_chunks))
    else:
        return int(any(gt == chunk for chunk in cleaned_chunks))

# ------------------- EIR -------------------

def eir_metric(gt_sentence, chunks):
    gt_sentences = split_into_sentences(gt_sentence)
    retrieved_text = " ".join(chunks)
    retrieved_sentences = split_into_sentences(retrieved_text)

    total_words = sum(count_words(chunk) for chunk in chunks)
    if total_words == 0:
        return 0.0

    matched_words = 0
    for gt in gt_sentences:
        gt_norm = normalize_text(gt)
        for rt in retrieved_sentences:
            if gt_norm in normalize_text(rt):
                matched_words += count_words(gt)
                break
    return matched_words / total_words

# ------------------- NDCG -------------------

def ndcg_metric(gt_sentence, chunks, k):
    gt_sentences = set(normalize_text(s) for s in split_into_sentences(gt_sentence))
    relevance_scores = []

    for chunk in chunks[:k]:
        chunk_sentences = set(normalize_text(s) for s in split_into_sentences(chunk))
        rel = int(any(gt in chunk_sentences for gt in gt_sentences))
        relevance_scores.append(rel)

    dcg = sum(rel / math.log2(i + 2) for i, rel in enumerate(relevance_scores))
    idcg = sum(1 / math.log2(i + 2) for i in range(min(len(gt_sentences), k)))

    return dcg / idcg if idcg else 0.0

# ------------------- Ana Değerlendirme Döngüsü -------------------

def evaluate_all(df, embed_model, top_k):
    recall_total = eir_total = ndcg_total = 0
    num_queries = 0

    for _, row in df.iterrows():
        source_collection = os.path.splitext(row['filename'])[0]  # Örn: wiki_03
        question = row['question']
        gt_sentence = row['sentence']

        chunks = retrieve_chunks(
        collection_name="wiki_combined", 
        embed_model=embed_model,
        query=question,
        k=top_k
        )

        if not chunks:
            print(f"'{source_collection}' kaynaklı chunk alınamadı.")
            continue

        recall = recall_metric(gt_sentence, chunks)
        eir = eir_metric(gt_sentence, chunks)
        ndcg = ndcg_metric(gt_sentence, chunks, k=top_k)

        print(f"\n🔍 {question}")
        print(f"Recall: {recall} | EIR: {eir:.4f} | nDCG@{top_k}: {ndcg:.4f}")

        recall_total += recall
        eir_total += eir
        ndcg_total += ndcg
        num_queries += 1

    print("\n===== TOPLU SONUÇLAR =====")
    print(f"Ortalama Recall: {recall_total / num_queries:.4f}")
    print(f"Ortalama EIR: {eir_total / num_queries:.4f}")
    print(f"Ortalama nDCG@{top_k}: {ndcg_total / num_queries:.4f}")
# ------------------- Kullanım -------------------

if __name__ == "__main__":
    csv_path = "C:\\Users\\memo_\\Desktop\\Projeler\\AutoRAG-End-to-End-RAG-Framework-with-Turkish-Embedding-Evaluation\\wiki_with_full_qa_random.csv"
    top_k = 5
    nrows = 45
    embed_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

    df = pd.read_csv(csv_path, nrows=nrows)
    evaluate_all(df, embed_model, top_k)


C:\Users\memo_\AppData\Local\Temp\ipykernel_7136\466041145.py:12: UserWarning: Qdrant client version 1.14.3 is incompatible with server version 1.12.6. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  return QdrantClient(
C:\Users\memo_\AppData\Local\Temp\ipykernel_7136\466041145.py:35: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(



🔍 Sosyalist İktidar Partisi'nin adı ne zaman ve ne olarak değişmiştir?
Recall: 1 | EIR: 0.0273 | nDCG@5: 1.0000

🔍 Recep Tayyip Erdoğan kimdir?
Recall: 1 | EIR: 0.0180 | nDCG@5: 0.4693

🔍 Arkeoloji nedir?
Recall: 1 | EIR: 0.0202 | nDCG@5: 1.0000

🔍 Enis Batur kimdir?
Recall: 1 | EIR: 0.0144 | nDCG@5: 0.6131

🔍 Laparoskopi nedir?
Recall: 1 | EIR: 0.0717 | nDCG@5: 1.0000

🔍 20. yüzyıl hangi tarih aralığını kapsar?
Recall: 0 | EIR: 0.0196 | nDCG@5: 0.0000

🔍 1944 nedir?
Recall: 1 | EIR: 0.1200 | nDCG@5: 1.0000

🔍 Erzurum neresidir?
Recall: 1 | EIR: 0.0077 | nDCG@5: 1.0000

🔍 Ekonomide esneklik (elastikiyet) ne anlama gelir?
Recall: 1 | EIR: 0.0289 | nDCG@5: 1.0000

🔍 Çankırı nerede bulunur?
Recall: 1 | EIR: 0.0086 | nDCG@5: 0.5000

🔍 Bulgaristan nerede yer alır?
Recall: 1 | EIR: 0.0096 | nDCG@5: 1.0000

🔍 Maldivler nerede bulunur ve kaç adadan oluşur?
Recall: 1 | EIR: 0.0176 | nDCG@5: 1.0000

🔍 Jeremy Bentham kimdir?
Recall: 1 | EIR: 0.0107 | nDCG@5: 1.0000

🔍 Gelir dağılımı neyi gösteri